In [ ]:
#EM TESTES
#VERSAO COM VGG16

In [ ]:
import cv2
import numpy as np
import os
import pandas as pd
import requests
from io import BytesIO
import tensorflow as tf
from tensorflow.keras.layers import (
    Input,
    Embedding,
    LSTM,
    Attention,
    Dropout,
    Dense,
    GlobalAveragePooling2D,
    GlobalAveragePooling1D,
    concatenate,
)
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import webcolors
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from typing import Tuple, Dict, List, Optional, Union
from tensorflow.keras.layers import Flatten



import warnings
warnings.filterwarnings("ignore")


class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, use_scale=True, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        self.use_scale = use_scale

    def build(self, input_shape):
        self.w_q = self.add_weight(
            name='w_q',
            shape=(input_shape[-1], input_shape[-1]),
            initializer='uniform',
            trainable=True
        )
        self.w_k = self.add_weight(
            name='w_k',
            shape=(input_shape[-1], input_shape[-1]),
            initializer='uniform',
            trainable=True
        )
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        q = tf.matmul(inputs, self.w_q)
        k = tf.matmul(inputs, self.w_k)
        v = inputs
        if self.use_scale:
            q /= tf.sqrt(tf.cast(tf.shape(k)[-1], tf.float32))
        attn_scores = tf.matmul(q, k, transpose_b=True)
        attn_scores = tf.nn.softmax(attn_scores, axis=-1)
        output = tf.matmul(attn_scores, v)
        return output

    def compute_output_shape(self, input_shape):
        return input_shape



def identify_colors(image: np.ndarray) -> List[Dict[str, str]]:
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    small_image = cv2.resize(image_rgb, (100, 100))
    color_list = small_image.reshape(-1, small_image.shape[-1])
    color_info = []
    for color in color_list:
        try:
            closest_color = webcolors.rgb_to_name(color)
            color_hex = "#{:02x}{:02x}{:02x}".format(color[0], color[1], color[2])
            color_info.append({'name': closest_color, 'hex': color_hex})
        except ValueError:
            pass

    unique_colors = []
    for color in color_info:
        if color not in unique_colors:
            unique_colors.append(color)
    return unique_colors

def process_image(image_path: str, resize: bool = True) -> Tuple[Optional[np.ndarray], Optional[str]]:
    if image_path.startswith('http'):
        response = requests.get(image_path)
        if response.status_code == 200:
            img_data = BytesIO(response.content)
            img = cv2.imdecode(np.frombuffer(img_data.read(), np.uint8), 1)
            if img is not None:
                if resize:
                    img = cv2.resize(img, (300, 300))
                return img, image_path
    elif os.path.exists(image_path):
        img = cv2.imread(image_path)
        if img is not None:
            if resize:
                img = cv2.resize(img, (300, 300))
            return img, image_path
    return None, None


def calculate_text_similarity(text1: str, text2: str) -> float:
    set1 = set(text1.lower().split())
    set2 = set(text2.lower().split())
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union) if len(union) > 0 else 0.0



def calculate_image_similarity(image1: np.ndarray, image2: np.ndarray) -> float:
    vgg16 = VGG16(weights='imagenet', include_top=False)

  
    image1 = cv2.resize(image1, (224, 224))
    image1 = np.expand_dims(image1, axis=0)
    image1 = preprocess_input(image1)

    image2 = cv2.resize(image2, (224, 224))
    image2 = np.expand_dims(image2, axis=0)
    image2 = preprocess_input(image2)

  
    features1 = vgg16.predict(image1)
    features2 = vgg16.predict(image2)

    
    similarity = np.dot(features1.flatten(), features2.flatten())
    return similarity


def compare_images_with_reference(reference_image: np.ndarray, image_list: List[np.ndarray]) -> List[float]:
    similarities = []
    for image in image_list:
        similarity = calculate_image_similarity(reference_image, image)
        similarities.append(similarity)
    return similarities

def find_matching_category(avaliacao_nome: str, treino: pd.DataFrame) -> str:
    max_similarity = 0.0
    matching_category = ""
    for _, row_treino in treino.iterrows():
        nome_treino = row_treino['nome']
        similarity = calculate_text_similarity(avaliacao_nome, nome_treino)
        if similarity > max_similarity:
            max_similarity = similarity
            matching_category = row_treino['Categoria']
    return matching_category


def create_info_dict(nome: str, imagem: str, categoria: str, probabilidade: float) -> Dict[str, Union[str, str, float]]:
    info_dict = {
        'Nome Produto Avaliação': nome,
        'Imagem': imagem,
        'Categoria': categoria,
        'Probabilidade': probabilidade,
    }
    return info_dict


def extract_image_features(image_path: str) -> np.ndarray:
    img, _ = process_image(image_path, resize=True)
    if img is not None:
        # Carregue o modelo MobileNetV2 pré-treinado
        base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                                       include_top=False,
                                                       weights='imagenet')
        base_model.trainable = False  
         # Correção: use (224, 224, 3) em vez de (300, 300, 3)
        input_image = Input(shape=(224, 224, 3)) 
        mobilenet_output = base_model(input_image) 

     
        image_features = Flatten()(mobilenet_output)  
        return image_features
    else:
        return None


root_dir: str = r'D:\imagesapp\images'  
treino: Optional[pd.DataFrame] = pd.read_csv(os.path.join(root_dir, 'images.csv'), sep=";")
avaliacao: Optional[pd.DataFrame] = pd.read_csv(os.path.join(root_dir, 'avaliacao.csv'), sep=";")

if treino is not None and avaliacao is not None:
   
    tokenizer: Tokenizer = Tokenizer(oov_token="<OOV>")
    tokenizer.fit_on_texts(treino['nome'])

    texts_train_sequences: np.ndarray = tokenizer.texts_to_sequences(treino['nome'])
    max_sequence_length: int = 300
    texts_train_sequences = pad_sequences(texts_train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
    text_vocab_size: int = len(tokenizer.word_index) + 1

  
    image_features_list = []
    for _, row_avaliacao in avaliacao.dropna().iterrows():
        imagem_avaliacao = row_avaliacao['imagem']
        image_features = extract_image_features(imagem_avaliacao)
        image_features_list.append(image_features)

  
    image_features_array = np.array(image_features_list)


    input_text: tf.Tensor = Input(shape=(max_sequence_length,), dtype=tf.int32)
    text_model: tf.keras.Model = tf.keras.Sequential([
        Embedding(input_dim=text_vocab_size, output_dim=256, input_length=max_sequence_length),
        LSTM(512, return_sequences=True),
        LSTM(512, return_sequences=True),
        AttentionLayer(use_scale=True),
        GlobalAveragePooling1D(),
    ])
    text_output: tf.Tensor = text_model(input_text)

  
    combined: tf.Tensor = concatenate([image_features_array, text_output])
    x: tf.Tensor = Dense(2048, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(combined)
    x = Dropout(0.6)(x)
    output: tf.Tensor = Dense(1, activation='sigmoid')(x)

    model: tf.keras.Model = Model(inputs=[input_image, input_text], outputs=output)

    for layer in base_model.layers:
        layer.trainable = True

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    avaliacao_info_list: List[Dict[str, Union[str, str, float]]] = []

 
    reference_image = treino.iloc[0]['imagem']

    for _, row_avaliacao in avaliacao.dropna().iterrows():
        nome_avaliacao = row_avaliacao['nome']
        imagem_avaliacao = row_avaliacao['imagem']

        img, _ = process_image(imagem_avaliacao)

        if img is not None:
            
            similarity = calculate_image_similarity(img, reference_image)

            texto_exemplo: str = nome_avaliacao
            texto_exemplo_sequence: np.ndarray = tokenizer.texts_to_sequences([texto_exemplo])
            texto_exemplo_sequence = pad_sequences(texto_exemplo_sequence, maxlen=max_sequence_length, padding='post', truncating='post')
            previsao_probabilidades: np.ndarray = model.predict([np.array([img]), texto_exemplo_sequence])

            probabilidade = previsao_probabilidades[0][0]

            avaliacao_categoria = nome_avaliacao
            categoria_prevista = find_matching_category(avaliacao_categoria, treino)

            info_dict: Dict[str, Union[str, str, float]] = create_info_dict(
                nome_avaliacao, imagem_avaliacao, categoria_prevista, probabilidade
            )

            print("Informações do Produto Avaliação:")
            for key, value in info_dict.items():
                print(f"{key}: {value}")

            cores_identificadas: List[Dict[str, str]] = identify_colors(img)
            for color_info in cores_identificadas:
                print(f"Nome da Cor: {color_info['name']}")
                print(f"Código Hexa: {color_info['hex']}")

            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            plt.title('Imagem do Produto')
            plt.axis('off')
            plt.show()

            print(f"Nome do Produto Avaliação: {nome_avaliacao}")
            print(f"Probabilidade: {probabilidade}")

            avaliacao_info_list.append(info_dict)

        else:
            print(f"Erro ao processar imagem de '{nome_avaliacao}'")